In [1]:
import numpy as np
from symmer.symplectic import (AntiCommutingOp, PauliwordOp, 
                               random_anitcomm_2n_1_PauliwordOp)

from symmer.symplectic.anticommuting_op import conjugate_Pop_with_R

# 1. Given an anticommuting set of operators

In [2]:
# AC_dict = {'ZZZ':2, 'XXX':1, 'YYY':2}
# test_op = AntiCommutingOp.from_dictionary(AC_dict)


### randomly generate anticommuting operator
nq = 100
test_op = AntiCommutingOp.from_PauliwordOp(random_anitcomm_2n_1_PauliwordOp(nq))
test_op.coeff_vec = (test_op.coeff_vec.real).astype(complex)
test_op.n_terms

201

In [3]:
test_op

 0.228+0.000j ZXYYIXXYZIZXYXIIZYZYYXXZXZYIYZIIYYIIIIYXZIIXIZIIXYYYXYIZZIYZXYXIXZYYXYXXZZIYYIXIYYZXXZXZYIZYIYXIZZYY +
-0.567+0.000j XIXXIIXXZZIXZIZYZZZXYIIXIYZXIYZXXZZZYZXXIXYYZIYYYXYZYXXIZXYIXIIXXIXIZXIIZXZIZIYYZIZZXIYZXZXYXZYZYIZY +
 0.713+0.000j XIXIXXXZIXXXIIZZZXZYXZYZIYZXIYXIYZZIZIZZYYZYZIXXIYZYXYZZYZYIIYZYZIXXXZZIZYXZZZIZXXZYIIIYZZIXZYXZYYII +
 0.157+0.000j YZYZYXZZIZXZIYXZIIIXZYXYIIXIIIXZIZXYXXZZIXIIYYZIXIYXXIXYYXXIYZXYXXZXYZZXXZXYXZXYZIXZIXIZYXIIZIXXIIZZ +
 0.161+0.000j IZXZXZIXIIXIIXIZZIYYXZYIZZYYXXZZXIIYYIYZYIIXIYYYXZZZXIZZYZZYZXIYYXIIZXXZIXZIYYYZXYYYIYZZIYZIIZZXXXZZ +
-0.432+0.000j YYZXYZYIIIYXIIXXYIYIYIXYIYIXXXIYIZXXIXYIZIZXXXIYYYXZXIZYZXYIYYZYXZIIIZXXXZIZZZXIYIZXZXYYZIIXXXYXXZXY +
-0.979+0.000j IZXXIIYXXYYXYXYIZZXYZXZIIXXIYIIXIXYYXIZXIYIIZXXZXYIIXXYYIYXXYXZZZXZIXIZXIIZXYZXYIIZYIIXZIIXZIZXZZIXY +
-0.519+0.000j XXXZYYYYZIZZIZIIXXIZYXIZYIXYZYIXZIIYZZIYYZIXZYZIYZZIZIXIZXXIIZYYXYIZZZXYZXIZXYZXZIXIXXXZZYIIIXXXZXIX +
-0.277+0.000j ZZIXZZYXIXXXIZXYIZZXXYXXZIIYYIYXXZIYXXXYZIZYXZYIYX

# 2. Use unitary partitioning to find unitary mapping to a single term in the linear combination

For full details see the following papers:
- [PAPER](https://arxiv.org/abs/1908.08067)
- [PAPER](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033195)

TWO methods:
1. linear combination of unitaries (LCU)
2. a sequence of (exponentiated pauli) rotations

## 2.1 LCU method

In [4]:
test_op

 0.228+0.000j ZXYYIXXYZIZXYXIIZYZYYXXZXZYIYZIIYYIIIIYXZIIXIZIIXYYYXYIZZIYZXYXIXZYYXYXXZZIYYIXIYYZXXZXZYIZYIYXIZZYY +
-0.567+0.000j XIXXIIXXZZIXZIZYZZZXYIIXIYZXIYZXXZZZYZXXIXYYZIYYYXYZYXXIZXYIXIIXXIXIZXIIZXZIZIYYZIZZXIYZXZXYXZYZYIZY +
 0.713+0.000j XIXIXXXZIXXXIIZZZXZYXZYZIYZXIYXIYZZIZIZZYYZYZIXXIYZYXYZZYZYIIYZYZIXXXZZIZYXZZZIZXXZYIIIYZZIXZYXZYYII +
 0.157+0.000j YZYZYXZZIZXZIYXZIIIXZYXYIIXIIIXZIZXYXXZZIXIIYYZIXIYXXIXYYXXIYZXYXXZXYZZXXZXYXZXYZIXZIXIZYXIIZIXXIIZZ +
 0.161+0.000j IZXZXZIXIIXIIXIZZIYYXZYIZZYYXXZZXIIYYIYZYIIXIYYYXZZZXIZZYZZYZXIYYXIIZXXZIXZIYYYZXYYYIYZZIYZIIZZXXXZZ +
-0.432+0.000j YYZXYZYIIIYXIIXXYIYIYIXYIYIXXXIYIZXXIXYIZIZXXXIYYYXZXIZYZXYIYYZYXZIIIZXXXZIZZZXIYIZXZXYYZIIXXXYXXZXY +
-0.979+0.000j IZXXIIYXXYYXYXYIZZXYZXZIIXXIYIIXIXYYXIZXIYIIZXXZXYIIXXYYIYXXYXZZZXZIXIZXIIZXYZXYIIZYIIXZIIXZIZXZZIXY +
-0.519+0.000j XXXZYYYYZIZZIZIIXXIZYXIZYIXYZYIXZIIYZZIYYZIXZYZIYZZIZIXIZXXIIZYYXYIZZZXYZXIZXYZXZIXIXXXZZYIIIXXXZXIX +
-0.277+0.000j ZZIXZZYXIXXXIZXYIZZXXYXXZIIYYIYXXZIYXXXYZIZYXZYIYX

In [5]:
# s_index gives index in symplectic matrix of Pauli term to be reduced too
# if NOT defined then s_index is automatically chosen to be least Pauli dense term

Ps_LCU, rotations_LCU, gamma_l_LCU, normed_clique_LCU = test_op.unitary_partitioning(up_method='LCU',
                                                                                    s_index=2)

# Ps_LCU = term that is reduced too
# rotations_LCU = PauliwordOp of LCU unitary that does this
# gamma_l_LCU = normalization factor of clique
# normed_clique_LCU = normalized anticommuting operator

In [6]:
Ps_LCU

 1.000+0.000j XIXIXXXZIXXXIIZZZXZYXZYZIYZXIYXIYZZIZIZZYYZYZIXXIYZYXYZZYZYIIYZYZIXXXZZIZYXZZZIZXXZYIIIYZZIXZYXZYYII

In [7]:
print(rotations_LCU)

 0.724+0.000j IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII +
-0.000-0.011j IZZIZZIZYXYXIXIZYZZXXXXZZYZYXZYXYYIIZXZXZXZZXXYIIZYIZXZIXXIIXIYIXXIIYYXXXYIIZYZYXIXZYXYYZXZIIYXZYXZZ +
-0.000-0.055j IZZIIZIYIZYXZYXYZIIZXZZIXXXXXZXZYYXIIXYXIXZIIIYXIYZIYXZXIIZXYZZIZXYIIYYIYYXYZYZIIZZYYZYZYXXIXXYZIZIY +
 0.000+0.007j ZZZZZIYIIYIYIYYIZXZZYXZXIYYXIYIZYIYYYXIIYZZYXYYXXYXZIYYXIYZIYXYIYXYIZIIXYXIXYIXXYXYXIXIXXYIXIYIYYYZZ +
-0.000-0.034j IZZZIZXZXZZIZYIXXXZIXYIXIZZIZXYIYXIZZYYYXYIXIIZZZZYZZYYXXZXYIXIZIZZZZZYYYYYYIZXYIIXIZZXYYIZXYXZXXYIY +
-0.000-0.081j IIIIIYZZIXYXIXYIZXXZXIIZYIXZXZYIYYZIXIXYIZZZZZXIXZZXYXYYXXXXZXYIXZZZYZIXXZIZYYYZYXYYXZYXZYIIIIXXYZIY +
-0.000-0.074j ZIIIZXIIXIXZXIXIIIIXZYYYZIXZZXXYIZXXIZZIZYIYZIXZYZXIIIZZYIZZXXIZZXYIYIYYXIYIIXYIIIXZYXZYIIZIYZYYXZZZ +
-0.000-0.003j ZIIIIXIXZYXZIYXIIYYIZIIXXZZIZXYIIZIXZZYIIYIXYXXYYYIIXIZYZXIYYYXZXXIIYXIYXIZYIXYYXZZYYZZZXIXIZIXYZXIY +
-0.000-0.022j ZIIZZXXXIIIYZXXYZIYXZZXZYYZYXZYXIIIYZIZZYXZXYXXIIZ

In [8]:
# clique normalized by gamma_l
test_op == normed_clique_LCU.multiply_by_constant(gamma_l_LCU)

True

In [10]:
# check reduction
%timeit rotations_LCU * normed_clique_LCU * rotations_LCU.dagger

143 ms ± 5.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [53]:
def modified_mult(A, B):
    if A.n_terms > B.n_terms:
        return (B.conjugate() * A.conjugate()).conjugate()
    else:
        return A * B

In [55]:
%timeit modified_mult(rotations_LCU, modified_mult(normed_clique_LCU, rotations_LCU.conjugate))

141 ms ± 7.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
modified_mult(rotations_LCU, modified_mult(normed_clique_LCU, rotations_LCU.conjugate)) == rotations_LCU * normed_clique_LCU * rotations_LCU.conjugate

True

In [ ]:
# other method to perform conjugation
%timeit conjugate_Pop_with_R(normed_clique_LCU, rotations_LCU)

In [45]:
rotations_LCU.n_terms

201

In [9]:
len(rotations_SeqRot)

200

In [46]:
# note what happens if rotation performed without normalization
print('normaliziation factor:', gamma_l_LCU)

rotations_LCU * test_op * rotations_LCU.conjugate

normaliziation factor: 14.796559890619632


 14.797+0.000j YIXZYIZYXIYIXXXIZXXZXXXIZYIXZIXYYIXXZZZXIYIIIZXZYXYIZZYIZIIYIIXZZIZYXIYZXZXZIXYIYZXXXIIZYYIYIXYZYYYZ

In [47]:
conjugate_Pop_with_R(normed_clique_LCU, rotations_LCU)

KeyboardInterrupt: 

## 2.2 Sequence of Rotations method

In [8]:
# Ps_SeqRot = term that is reduced too
# rotations_SeqRot = list of PauliwordOp rotations. List of tuples of (P, angle_of_rotation)
# gamma_l_SeqRot = normalization factor of clique
# normed_clique_SeqRot = normalized anticommuting operator
(Ps_SeqRot, rotations_SeqRot, 
 gamma_l_SeqRot, normed_clique_SeqRot) = test_op.unitary_partitioning(up_method='seq_rot',
                                                                      s_index=None)

In [37]:
print(rotations_SeqRot)

[( 1.000-0.000j IZYYXYXXYYIYZIIZYYZZXIIXYIIYXXZXZZYYYXZXXXXYIXIIYXXIZXYXZZZIZYZXZYXZYYXYZXYZXZXYYYIIXXXZZYYZZXXXIZIX, (-0.9639086986511329+0j)), ( 1.000-0.000j XIXYXIYIZXZZIYYXYXIXZXYYYYIXIYXIZZXZZXYXXZZYYXIYXIIZXIZXYZYXYZIIXYZZIXYYIYXXYXYZZYZZIIXIXIIYZIIZYYZZ, (-0.17767892234310637+0j)), ( 1.000-0.000j ZXIXXZIZZZYXIXXZXIYXZIIXYXIIXYYZXIYXIXZXYYXZYIXXXIIYYIYIXIIZYYXXIXZYXZYZZZZZXZXIIXYXIZIXIIIZYZZXIZXI, (-0.9712829226214318+0j)), ( 1.000-0.000j XYXIZXYYXIIXXZYIXYXYIZYXXYIZIYZXYXXYIXYXZYZZIXXIXXIXZIYZYZXZZZYYYYXIIYXZZXZIZXXYXYXXXYXIYZYXIZXYXIZY, (-0.1584655474450426+0j)), ( 1.000-0.000j IXXYZIIXYXZIXZZYIXIYYIYIZIXIXXYXXIZXIYIIXZIIXYXIZYIYYIIIZIXXXZIIYZYXYYYZZXXIZZXXYXZXIZIZZIXIXZXXXYXZ, (-0.7643730848722909+0j)), ( 1.000-0.000j XYXXYIIIZZXZYXZXXYYZZXIIXIZXZIZIZXIXIYXYZZXIZYZXYYIIZIZZZIZXIYIIZZZYZIXZIYZIIYZZIIYXIIZZYZIXXYYZYYXX, (0.5021320767491653-0j)), ( 1.000-0.000j XYYZYZXYXXXZXZIXIYIXZZYYZIIYZIIXIIXZXXYZXZXYXIYYYXIXXIXXYYZXZZXZXYYZZZXZZXYYZYIXYZZYIZZYIZIIXYYZXIYY, (0.391052606334286

In [38]:
# clique normalized by gamma_l
test_op == normed_clique_SeqRot.multiply_by_constant(gamma_l_SeqRot)

True

In [39]:
# apply sequence of rotations on anticommuting operator (normalized)
normed_clique_SeqRot.perform_rotations(rotations_SeqRot)

 1.000+0.000j IIIZYYYIXZXYYYYXYXIIYZYYYIYXXIZZXXIIZYYIYXYYXZIZIYXXZXYIIIXZXIYXYIYXZYXYYYXZXIYIYIIIYXXZZZIZXZYXYIZX

In [40]:
# without normalization
test_op.perform_rotations(rotations_SeqRot)

 14.034+0.000j IIIZYYYIXZXYYYYXYXIIYZYYYIYXXIZZXXIIZYYIYXYYXZIZIYXXZXYIIIXZXIYXYIYXZYXYYYXZXIYIYIIIYXXZZZIZXZYXYIZX +
 0.000+0.000j ZYXXZZYXYXIIIZZZYZIIXZZZXYIYXYXYIXIXYIXIIIXYXIXYXXZXYYZIYYIXYXYXIYXXYXXXIZZYYYZXIZYXXIZXYXZIZIIZYYZX +
 0.000+0.000j YZXXXZIIZXIZXYYIIZIZIYZXIIXXXIZYZZZYIYYIXYYIYZIIZZZIXYIXZXZXXXYXZZIZIYXIZIZXXIIZXIYZXZYYIXYYXZXXXYXX

In [41]:
rotations_SeqRot

[( 1.000-0.000j IZYYXYXXYYIYZIIZYYZZXIIXYIIYXXZXZZYYYXZXXXXYIXIIYXXIZXYXZZZIZYZXZYXZYYXYZXYZXZXYYYIIXXXZZYYZZXXXIZIX,
  (-0.9639086986511329+0j)),
 ( 1.000-0.000j XIXYXIYIZXZZIYYXYXIXZXYYYYIXIYXIZZXZZXYXXZZYYXIYXIIZXIZXYZYXYZIIXYZZIXYYIYXXYXYZZYZZIIXIXIIYZIIZYYZZ,
  (-0.17767892234310637+0j)),
 ( 1.000-0.000j ZXIXXZIZZZYXIXXZXIYXZIIXYXIIXYYZXIYXIXZXYYXZYIXXXIIYYIYIXIIZYYXXIXZYXZYZZZZZXZXIIXYXIZIXIIIZYZZXIZXI,
  (-0.9712829226214318+0j)),
 ( 1.000-0.000j XYXIZXYYXIIXXZYIXYXYIZYXXYIZIYZXYXXYIXYXZYZZIXXIXXIXZIYZYZXZZZYYYYXIIYXZZXZIZXXYXYXXXYXIYZYXIZXYXIZY,
  (-0.1584655474450426+0j)),
 ( 1.000-0.000j IXXYZIIXYXZIXZZYIXIYYIYIZIXIXXYXXIZXIYIIXZIIXYXIZYIYYIIIZIXXXZIIYZYXYYYZZXXIZZXXYXZXIZIZZIXIXZXXXYXZ,
  (-0.7643730848722909+0j)),
 ( 1.000-0.000j XYXXYIIIZZXZYXZXXYYZZXIIXIZXZIZIZXIXIYXYZZXIZYZXYYIIZIZZZIZXIYIIZZZYZIXZIYZIIYZZIIYXIIZZYZIXXYYZYYXX,
  (0.5021320767491653-0j)),
 ( 1.000-0.000j XYYZYZXYXXXZXZIXIYIXZZYYZIIYZIIXIIXZXXYZXZXYXIYYYXIXXIXXYYZXZZXZXYYZZZXZZXYYZYIXYZZYIZZYIZIIXYYZXIYY,


# 3. Further work

Some methods require applying unitary partitioning operator (unitary of LCU or seq_rot) to other combinations of Pauli operators (such as a Hamiltonian)

For examples see:
- [PAPER1](https://arxiv.org/abs/2011.10027) 
- [PAPER2](https://arxiv.org/abs/2204.02150)
- [PAPER3](https://arxiv.org/abs/2207.03451)

This subsection looks at the scaling of the different methods

In [11]:
### Generate a random linear combination of Pauli operators (on same no. of qubits as rotations above)

n_terms= 100
# n_terms= 500
# n_terms= 12
op = PauliwordOp.random(test_op.n_qubits, n_terms)

# op = PauliwordOp.haar_random(test_op.n_qubits)
op.n_terms

100

In [14]:
## SeqRot scaling
print('max possible P terms on defined qubits:', 4**rotations_LCU.n_qubits)
print('max terms SeqRot:', 2**rotations_LCU.n_terms * op.n_terms)

max possible P terms on defined qubits: 1606938044258990275541962092341162602522202993782792835301376
max terms SeqRot: 321387608851798055108392418468232520504440598756558567060275200


In [15]:
op.perform_rotations(rotations_SeqRot).n_terms

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tweaving/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_183227/1624288337.py", line 1, in <module>
    op.perform_rotations(rotations_SeqRot).n_terms
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/symplectic/base.py", line 623, in perform_rotations
    op_copy = op_copy._rotate_by_single_Pword(pauli_rotation, angle).cleanup()
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/symplectic/base.py", line 282, in cleanup
    *symplectic_cleanup(
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/utils.py", line 137, in symplectic_cleanup
    reduced_symp_matrix = sorted_terms[mask_unique_terms]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/tweaving/.local/lib/python3.8/site-pac


KeyboardInterrupt



In [19]:
for i in range(1, 50):
    print(op.perform_rotations(rotations_SeqRot[:i]).n_terms)

155
232
350
526
785
1149
1733
2555
3806
5834
8723
12612
19190
28993
43020
66114
97685
145910
210403
317273
467762
707897
1071419
1592002
2391669
3641773
5413622


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tweaving/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_183227/4133640676.py", line 2, in <module>
    print(op.perform_rotations(rotations_SeqRot[:i]).n_terms)
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/symplectic/base.py", line 623, in perform_rotations
    op_copy = op_copy._rotate_by_single_Pword(pauli_rotation, angle).cleanup()
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/symplectic/base.py", line 606, in _rotate_by_single_Pword
    anticom_part = (anticom_self.multiply_by_constant(np.cos(angle)) +
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/symplectic/base.py", line 317, in __add__
    return PauliwordOp(P_symp_mat_new, P_new_coeffs).cleanup()
  File "/home/tweaving/anaconda3/lib/python3.8/site-packages/symmer/symplectic/base.py", line 282,

TypeError: object of type 'NoneType' has no len()

In [13]:
## LCU scaling
print('max terms on qubits:', 4**rotations_LCU.n_qubits)
print('max terms LCU:', rotations_LCU.n_terms**2 * op.n_terms)

max terms on qubits: 1606938044258990275541962092341162602522202993782792835301376
max terms LCU: 4040100


In [12]:
# manual multiplication
(rotations_LCU * op * rotations_LCU.conjugate).n_terms

1004316

In [ ]:
rot

**Important**: note how LCU method results in **fewer** terms!

- SeqRot method increases terms as ~$2^{|U|}$
- LCU method increases terms as  ~$|U|^{2}$

where $|U|$ is the number of Pauli terms in the operator performing unitary partitioning.

for further details see [PAPER](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033195)